# Wizualizacja poziomu zanieczyszczeń w Polsce

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv("Stats_2000-2023.csv", header = 0, skiprows = [1], encoding="ansi", delimiter=";")

df = df.loc[df["Czas uśredniania"] == "24g"] # Wybieramy tylko przedział pomiaru 24h
df = df.loc[df["Rok"] >= 2010] # Wybieramy tylko dane od 2010 roku


Chcemy wybrać dwie stacje, które mają dane dla każdego roku z przedziału [2010; 2023]

In [ ]:
# pv = pd.pivot_table(df, index = "Kod stacji", columns = "Rok", values = "Wskaźnik", aggfunc = "count")
# pv

Znaleźliśmy takie: 
- DsOsieczow21
- DsWalbrzWyso

In [ ]:
station1 = df[df["Kod stacji"] == "DsOsieczow21"]
station2 = df[df["Kod stacji"] == "DsWalbrzWyso"]
station1_measurement = station1.loc[:, "Średnia"].to_numpy().astype(str)
station2_measurement = station2.loc[:, "Średnia"].to_numpy().astype(str)

station1_measurement = np.char.replace(station1_measurement, ',', '.').astype(float)
station2_measurement = np.char.replace(station2_measurement, ',', '.').astype(float)

years = np.arange(start = 2010, stop = 2024)
plt.plot(years, station1_measurement, label = "DsOsieczow21", color = "blue")
plt.plot(years, station2_measurement, label = "DsWalbrzWyso", color = "red")

plt.ylim(10, 30)
plt.yticks(np.arange(10, 31, 2))

plt.xlabel("Rok")
plt.ylabel("Średnie natężenie PM2.5 [µg/m3]")
plt.title("Średnie natężenie PM2.5 w dwóch stacjach na przestrzeni lat 2010-2023")
plt.legend()
plt.show()


Jak widać średnie natężenie PM2.5 w obydwu stacjach pomiarowych maleje z roku na rok

Przedstawimy teraz rozkład średniego natężenia PM2.5 we wszystkich stacjach na przestrzeni lat 2010-2023

In [ ]:
df2 = df.loc[:, ["Kod stacji", "Średnia", "Rok"]] # Wybieramy pola, które nas interesują
plt.figure(figsize=(10, 50))
sns.boxplot(x = "Rok", y = "Kod stacji", data = df2)
plt.title("Rozkład średniego natężenia PM2.5 we wszystkich stacjach na przestrzeni lat 2010-2023")
plt.xlabel("Rok")
plt.ylabel("Stacja")
plt.show()

Jak widać, w większości stacji pomiarowych w Polsce średnie natężenie PM2.5 w Polsce ma bardzo duże różnice na przestrzeni lat

Zajmiemy się teraz liczbą przekroczeń normy PM2.5 według normy WHO (25 µg/m³)

In [ ]:
df1 = df.loc[:, ["Województwo", "Średnia"]]

# Zmieniamy format Średniej na float
tmp = df1["Średnia"].to_numpy().astype(str)
tmp = np.char.replace(tmp, ',', '.').astype(float)
df1["Średnia"] = tmp

filtered_df = df1[df1["Średnia"] > 25]
filtered_df = filtered_df.groupby("Województwo").count()

filtered_df.loc["zachodniopomorskie"] = 0
filtered_df.loc["warmińsko-mazurskie"] = 0

filtered_df.plot(kind = "bar")

plt.title("Liczba przekroczeń normy WHO stężenia PM2.5 dla poszczególnych województw")
plt.xlabel("Województwo")
plt.ylabel("Liczba przekroczeń")
plt.show()

Jak widać liczba przekroczeń normy WHO średniego stężenia PM2.5 jest niewielka w większości województw.
Wyróżniają się negatywnie jedynie województwa małopolskie oraz śląskie

In [ ]:
df = df[["Województwo", "Liczba pomiarów"]]
filtered_df2 = df.groupby("Województwo").sum()

joint_df = filtered_df.join(filtered_df2)

joint_df = joint_df["Średnia"] / joint_df["Liczba pomiarów"] * 100

joint_df.plot(kind = "bar")

plt.title("Procent pomiarów przekraczających normę WHO według województw")
plt.xlabel("Województwo")
plt.ylabel("Procent pomiarów przekraczających normę WHO")
plt.show()



Jak widać pomiarów średnich, które przekraczają normę WHO jest bardzo niewiele w całej Polsce